#### Imports


In [1]:
from zoish.feature_selectors.shap_selectors import ShapFeatureSelector
import pandas as pd
import xgboost
from sklearn.model_selection import train_test_split, KFold
import pandas as pd
from sklearn.pipeline import Pipeline
from feature_engine.imputation import MeanMedianImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score, make_scorer

2023-02-24 19:31:34,341 :: matplotlib :: matplotlib data path: /Users/hjavedani/Documents/zoish/.venv/lib/python3.10/site-packages/matplotlib/mpl-data
2023-02-24 19:31:34,354 :: matplotlib :: CONFIGDIR=/Users/hjavedani/.matplotlib
2023-02-24 19:31:34,362 :: matplotlib :: interactive is False
2023-02-24 19:31:34,363 :: matplotlib :: platform is darwin
2023-02-24 19:31:34,365 :: matplotlib :: loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', '_io', 'marshal', 'posix', '_frozen_importlib_external', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '_abc', 'abc', 'io', '__main__', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_virtualenv', '_distutils_hack', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib._abc', 'itertools', 'keyword', '_operator', 'operator', 'reprlib', '_colle

#### Example: Audiology (Standardized) Data Set
###### https://archive.ics.uci.edu/ml/datasets/Audiology+%28Standardized%29


#### Read data


In [2]:
urldata = "https://archive.ics.uci.edu/ml/machine-learning-databases/lymphography/lymphography.data"
urlname = "https://archive.ics.uci.edu/ml/machine-learning-databases/lung-cancer/lung-cancer.names"
# column names
col_names = [
    "class",
    "lymphatics",
    "block of affere",
    "bl. of lymph. c",
    "bl. of lymph. s",
    "by pass",
    "extravasates",
    "regeneration of",
    "early uptake in",
    "lym.nodes dimin",
    "lym.nodes enlar",
    "changes in lym.",
    "defect in node",
    "changes in node",
    "special forms",
    "dislocation of",
    "exclusion of no",
    "no. of nodes in",

]

data = pd.read_csv(urldata,names=col_names)
data.head()


,class,lymphatics,block of affere,bl. of lymph. c,bl. of lymph. s,by pass,extravasates,regeneration of,early uptake in,lym.nodes dimin,lym.nodes enlar,changes in lym.,defect in node,changes in node,special forms,dislocation of,exclusion of no,no. of nodes in
3,4,2,1,1,1,1,1,2,1,2,2,2,4,8,1,1,2,2
2,3,2,1,1,2,2,1,2,1,3,3,2,3,4,2,2,2,2
3,3,2,2,2,2,2,2,2,1,4,3,3,4,8,3,2,2,7
3,3,1,1,1,1,2,1,2,1,3,3,4,4,4,3,1,2,6
2,3,1,1,1,1,1,1,1,1,2,2,4,3,5,1,2,2,1


#### Define labels and train-test split


In [3]:


data.loc[(data["class"] == 1) | (data["class"] == 2), "class"] = 0
data.loc[data["class"] == 3, "class"] = 1
data.loc[data["class"] == 4, "class"] = 2
data["class"] = data["class"].astype(int)

#### Train test split


In [4]:
X = data.loc[:, data.columns != "class"]
y = data.loc[:, data.columns == "class"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33,  random_state=42
)

#### Define feature selector step 


In [5]:
shap_feature_selector_factory = (
    ShapFeatureSelector.shap_feature_selector_factory.set_model_params(
        X=X_train,
        y=y_train,
        verbose=0,
        random_state=0,
        estimator=xgboost.XGBClassifier(),
        estimator_params={
            "max_depth": [4, 5],
        },
        fit_params = {
            "sample_weight": None,
        },
        method="tunegridsearch",
        # if n_features=None only the threshold will be considered as a cut-off of features grades.
        # if threshold=None only n_features will be considered to select the top n features.
        # if both of them are set to some values, the threshold has the priority for selecting features.
        n_features=3,
        threshold = 0.4,
        list_of_obligatory_features_that_must_be_in_model=["defect in node"],
        list_of_features_to_drop_before_any_selection=["bl. of lymph. c"],
    ).set_shap_params(
        model_output="raw",
        feature_perturbation="interventional",
        algorithm="v2",
        shap_n_jobs=-1,
        memory_tolerance=-1,
        feature_names=None,
        approximate=False,
        shortcut=False,
    ).set_tunegridsearchcv_params(
        verbose=1,
        n_jobs=None,
        cv=KFold(3),
        early_stopping=None, 
        scoring=None, 
        refit=True, 
        error_score='raise', 
        return_train_score=False, 
        local_dir='~/ray_results', 
        name=None, 
        max_iters=1, 
        use_gpu=False, 
        loggers=None, 
        pipeline_auto_early_stop=True, 
        stopper=None, 
        time_budget_s=None, 
        measure_of_accuracy=make_scorer(f1_score, greater_is_better=True, average='macro'),
        mode=None
        )
)

#### Find feature type for later use


In [6]:
int_cols = X_train.select_dtypes(include=["int"]).columns.tolist()
print(int_cols)

['lymphatics', 'block of affere', 'bl. of lymph. c', 'bl. of lymph. s', 'by pass', 'extravasates', 'regeneration of', 'early uptake in', 'lym.nodes dimin', 'lym.nodes enlar', 'changes in lym.', 'defect in node', 'changes in node', 'special forms', 'dislocation of', 'exclusion of no', 'no. of nodes in']


#### Define pipeline

In [7]:
pipeline = Pipeline(
    [
        # int missing values imputers
        (
            "intimputer",
            MeanMedianImputer(imputation_method="median", variables=int_cols),
        ),
        ("sfsf", shap_feature_selector_factory),
        # classification model
        ("logistic", LogisticRegression()),
    ]
)

#### Run Pipeline


In [8]:
pipeline.fit(X_train, y_train.values.ravel())
y_pred = pipeline.predict(X_test)


2023-02-24 19:32:29,684	INFO tune.py:762 -- Total run time: 25.11 seconds (24.86 seconds for the tuning loop).


2023-02-24 19:32:30,282 :: dev :: The optimization will be based on make_scorer(f1_score, average=macro) metric!
2023-02-24 19:32:30,282 :: dev :: The optimization will be based on make_scorer(f1_score, average=macro) metric!
2023-02-24 19:32:30,446 :: dev :: this list of features also will be selectec! ['defect in node']
2023-02-24 19:32:30,446 :: dev :: this list of features also will be selectec! ['defect in node']
2023-02-24 19:32:30,447 :: dev :: this list of features  will be dropped! ['bl. of lymph. c']
2023-02-24 19:32:30,447 :: dev :: this list of features  will be dropped! ['bl. of lymph. c']


#### Check performance of the Pipeline


In [9]:

print("F1 score : ")
print(f1_score(y_test, y_pred,average='micro'))
print("Classification report : ")
print(classification_report(y_test, y_pred))
print("Confusion matrix : ")
print(confusion_matrix(y_test, y_pred))



F1 score : 
0.3877551020408163
Classification report : 
              precision    recall  f1-score   support

           0       0.45      0.71      0.56        21
           1       0.27      0.29      0.28        14
           2       0.00      0.00      0.00        14

    accuracy                           0.39        49
   macro avg       0.24      0.33      0.28        49
weighted avg       0.27      0.39      0.32        49

Confusion matrix : 
[[15  6  0]
 [ 9  4  1]
 [ 9  5  0]]


#### Get access to feature selector instance

In [10]:
print(ShapFeatureSelector.shap_feature_selector_factory.get_feature_selector_instance())